In [1]:
import pandas as pd
from pathlib import Path
from pprint import pprint
from datasets import load_dataset

## To dataset format

In [2]:
from datasets import Dataset, DatasetDict
from src.utils import FormatPrompt, DialogueManager


def last_turn_rag_conv(data, FP, system_prompt):

    last_turn_dataset = []
    for (conver_id, pattern, sampling_id), conversation in data.groupby(["conver_id", "pattern", "sampling_stergy"]):
        conv_id = f"{conver_id}_{pattern}_{sampling_id}"

        converstaions = []
        conv = eval(conversation.to_json(orient="records", force_ascii=False,))
        
        DM = DialogueManager(system_prompt=system_prompt, log_path=".cache/chat.log")    
        for i, items in enumerate(conv):
            Q = items['questions']
            A = items['answers']
            F = items['retrieved_facts']
            C = items['citations']

            ## INPUT ##
            user, meta_user = FP.format_input(Q, facts=F) 
            assistant, meta_assistant = FP.format_output(A, C)
            
            ## Keep Conv
            turn = DM.data() + [user, assistant]
            converstaions.append(turn)
            
            ## Add history
            meta_user = {k:v for k,v in meta_user.items() if k in ["role", "content"]}
            meta_assistant = {k:v for k,v in meta_assistant.items() if k in ["role", "content"]}
            DM.add([meta_user, meta_assistant]) 
            DM.save()
        
        question, answer = converstaions[-1][:-1], converstaions[-1][-1]
        last_turn_dataset.append({"id": f"{conv_id}_turn_{i}", "question": question, "answer": answer,})
        
    return Dataset.from_list(last_turn_dataset)

## Pipeline

In [8]:
system_prompt = Path(f"src/prompt/context_with_citations.md").read_text().strip()
FP = FormatPrompt("contextcitations")
# 1) INPUT:    user, meta_user = FP.format_input(q, facts=facts)
# 2) Extract:  FP.extract_output(out)
# 3) Format:   assistant, meta_assistant = FP.format_output(LLMs['answer'], LLMs['citations'])

# DM = DialogueManager(system_prompt=system_prompt, log_path=".cache/chat.log")    
# 2) GET:      turn = DM.data() + [user]  => 
# 3) ADD Q/A:  DM.add([meta_user, meta_assistant]) 
# DM.reset() and DM.save()

# Load data
dataset = load_dataset("weerayut/iq-data")
# test = dataset['test'].to_pandas()
# train = dataset['train'].to_pandas()
validation = dataset['validation'].to_pandas()

# Proc
# proc_test = last_turn_rag_conv(test, FP, system_prompt)
# proc_train = last_turn_rag_conv(train, FP, system_prompt)
proc_validation = last_turn_rag_conv(validation, FP, system_prompt)

# Upload data
# dataset = DatasetDict({"train": proc_train, "validation": proc_validation, "test": proc_test})
# dataset.push_to_hub("weerayut/iq-rag-lastturn", private=True)

## Check data

In [16]:
index = 3
question = proc_validation[index]['question']
answer = proc_validation[index]['answer']

proc_validation[index].keys()

dict_keys(['id', 'question', 'answer'])

In [17]:
pprint(question, sort_dicts=False, width=300)

[{'content': 'Answer only in non-duplicate JSON using provided context, include fact-ID citations, or return {"answer":"unknown","citations":[]} if unavailable.', 'role': 'system'},
 {'content': 'ความเครียดส่งผลต่อระบบภูมิคุ้มกันของร่างกายอย่างไร', 'role': 'user'},
 {'content': 'ความเครียดที่สะสมหรือเรื้อรังส่งผลให้ระบบภูมิคุ้มกันอ่อนแอลง เนื่องจากฮอร์โมนความเครียดที่หลั่งเพิ่มขึ้นจะยับยั้งการทำงานของระบบภูมิคุ้มกัน ทำให้ร่างกายป่วยง่ายขึ้น ติดเชื้อได้ง่ายและฟื้นตัวช้ากว่าปกติ', 'role': 'assistant'},
 {'content': 'ความเครียดเรื้อรังส่งผลต่อการทำงานของสมองส่วนใดมากที่สุด', 'role': 'user'},
 {'content': 'ความเครียดเรื้อรังส่งผลต่อการทำงานของสมองส่วนไฮโปธาลามัสและสมองส่วนที่เกี่ยวข้องกับความจำและสมาธิ โดยทำให้เกิดความผิดปกติในการหลั่งฮอร์โมนเจริญพันธุ์ ไข่ไม่ตก มีสมาธิสั้น และความจำแย่ลง เนื่องจากความเครียดมีผลโดยตรงต่อการทำงานของระบบประสาทส่วนกลาง', 'role': 'assistant'},
 {'content': 'คำถาม:\n'
             'ความเครียดเรื้อรังมีผลต่อฮอร์โมนคอร์ติซอลอย่างไร และส่งผลต่อระบบภูมิคุ้มกันอย่าง

In [18]:
pprint(answer, sort_dicts=False, width=300)

{'content': '{"answer": "ความเครียดเรื้อรังทำให้ร่างกายหลั่งฮอร์โมนคอร์ติซอลมากเกินไปจากต่อมหมวกไต โดยเริ่มต้นจากการที่ความเครียดกระตุ้นสมองส่วนไฮโปธาลามัส ซึ่งควบคุมการหลั่งฮอร์โมนในระบบประสาทและต่อมไร้ท่อ '
            'ส่งผลให้ระบบประสาทส่วนกลางสั่งการให้ต่อมใต้สมองและต่อมหมวกไตหลั่งฮอร์โมนคอร์ติซอลอย่างต่อเนื่อง ซึ่งฮอร์โมนนี้เมื่ออยู่ในระดับสูงเป็นเวลานานจะยับยั้งการทำงานของเซลล์ภูมิคุ้มกัน ทำให้ระบบภูมิคุ้มกันอ่อนแอลง ร่างกายติดเชื้อง่ายขึ้น และฟื้นตัวจากโรคได้ช้าลง", "citations": [1, 2, 3, 4, '
            '5]}',
 'role': 'assistant'}


## Apply with chat template

In [19]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B", trust_remote_code=True,)

# Build prompt
text = tokenizer.apply_chat_template(question, tokenize=False, add_generation_prompt=True,)
print(text)

<|im_start|>system
Answer only in non-duplicate JSON using provided context, include fact-ID citations, or return {"answer":"unknown","citations":[]} if unavailable.<|im_end|>
<|im_start|>user
ความเครียดส่งผลต่อระบบภูมิคุ้มกันของร่างกายอย่างไร<|im_end|>
<|im_start|>assistant
ความเครียดที่สะสมหรือเรื้อรังส่งผลให้ระบบภูมิคุ้มกันอ่อนแอลง เนื่องจากฮอร์โมนความเครียดที่หลั่งเพิ่มขึ้นจะยับยั้งการทำงานของระบบภูมิคุ้มกัน ทำให้ร่างกายป่วยง่ายขึ้น ติดเชื้อได้ง่ายและฟื้นตัวช้ากว่าปกติ<|im_end|>
<|im_start|>user
ความเครียดเรื้อรังส่งผลต่อการทำงานของสมองส่วนใดมากที่สุด<|im_end|>
<|im_start|>assistant
ความเครียดเรื้อรังส่งผลต่อการทำงานของสมองส่วนไฮโปธาลามัสและสมองส่วนที่เกี่ยวข้องกับความจำและสมาธิ โดยทำให้เกิดความผิดปกติในการหลั่งฮอร์โมนเจริญพันธุ์ ไข่ไม่ตก มีสมาธิสั้น และความจำแย่ลง เนื่องจากความเครียดมีผลโดยตรงต่อการทำงานของระบบประสาทส่วนกลาง<|im_end|>
<|im_start|>user
คำถาม:
ความเครียดเรื้อรังมีผลต่อฮอร์โมนคอร์ติซอลอย่างไร และส่งผลต่อระบบภูมิคุ้มกันอย่างไรต่อเนื่องจากสมองส่วนไฮโปธาลามัส?
Facts:
[1

## OPENAI

In [21]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-4o",
    messages=question
)

print(response.choices[0].message.content)

{"answer":"ความเครียดเรื้อรังสามารถเพิ่มระดับฮอร์โมนคอร์ติซอลจนส่งผลเสียต่อระบบภูมิคุ้มกันได้ เนื่องจากเมื่อสมองส่วนไฮโปธาลามัสถูกกระตุ้นจากความเครียด มันจะต่อมหมวกไตให้หลั่งคอร์ติซอลเป็นจำนวนมาก ฮอร์โมนคอร์ติซอลที่มากเกินไปจะทำให้ระบบภูมิคุ้มกันอ่อนแอลง เพิ่มความเสี่ยงต่อการติดเชื้อ และการฟื้นตัวจากบาดแผลช้าลง","citations":["1","5","8"]}
